In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
observation_start = pd.to_datetime("2010-05-10 18:00:00")
total_samples = 327
time_interval = pd.Timedelta(6, "h")
timestamps = pd.to_datetime(
    [observation_start + i * time_interval for i in range(total_samples)]
)
timestamps

DatetimeIndex(['2010-05-10 18:00:00', '2010-05-11 00:00:00',
               '2010-05-11 06:00:00', '2010-05-11 12:00:00',
               '2010-05-11 18:00:00', '2010-05-12 00:00:00',
               '2010-05-12 06:00:00', '2010-05-12 12:00:00',
               '2010-05-12 18:00:00', '2010-05-13 00:00:00',
               ...
               '2010-07-29 00:00:00', '2010-07-29 06:00:00',
               '2010-07-29 12:00:00', '2010-07-29 18:00:00',
               '2010-07-30 00:00:00', '2010-07-30 06:00:00',
               '2010-07-30 12:00:00', '2010-07-30 18:00:00',
               '2010-07-31 00:00:00', '2010-07-31 06:00:00'],
              dtype='datetime64[ns]', length=327, freq=None)

In [6]:
def prepare_data(
    script_name: str = None,
    data_folder_script_relative_path: str = "data/",
    data_file_type: str = ".dat",
) -> dict:
    """

    Args:
        script_name (str): name of running script. Defaults to None.
        data_folder_script_relative_path (str): data folder path relative to running script. Defaults to "data/".
        data_file_type (str): type of data file. Defaults to ".dat".

    Returns:
        dict: keys are integers from 1 to 2921 equivalent to site observation location ID and values are arrays of shape (16, 327).
            rows 0-14; GHG concentrations of tracers emitted from regions 1-15; row 15 is GHC concentrations of synthetic observations
            columns 0-326; GHC concentrations every 6 hour from May 10 to July 31, 2010  
    """

    script_path = os.path.abspath(path=script_name)
    # get the parent directory of the script (i.e., the scripts folder)
    scripts_folder = os.path.dirname(script_path)
    # move one level up to the parent directory of the scripts folder
    parent_folder = os.path.dirname(scripts_folder)
    # specify the relative path from the scripts folder to the data folder
    relative_path = data_folder_script_relative_path
    # construct the absolute path to the data folder
    data_folder = os.path.join(parent_folder, relative_path)
    # get a list of all files and directories in the folder
    file_list = os.listdir(data_folder)

    data_dict = {}
    # iterate over the files and process/import the .dat files
    for file_name in file_list:
        if file_name.endswith(data_file_type):
            # get observation site location id
            obs_site_loc_id = [int(num) for num in re.findall(r"\d+", file_name)][0]
            file_path = os.path.join(data_folder, file_name)
            with open(file_path, "r") as file:
                contents = file.read()
                # split the long string into lines
                rows = contents.splitlines()
                # 16 rows expected; rows 1-15 for 15 regions and row 16 for synthetic observations
                rows = rows[:16]
                # determine the number of columns from the first row
                num_columns = len(rows[0].split())
                assert (
                    num_columns == 327
                ), f"327 concentration columns are expected while {num_columns} columns were detected!"
                try:
                    # Load the data as a 2D float array, selecting a subset of columns
                    data_dict[obs_site_loc_id] = np.loadtxt(
                        rows, usecols=range(num_columns), dtype=float
                    )
                except ValueError as e:
                    print(f"Error processing file {file_name}: {e}")
    assert (
        len(data_dict.keys()) == 2921
    ), f"2921 observation sites are expected while {len(data_dict.keys())} sites were detected!"
    return data_dict

In [7]:
prepare_data("inverse_methods_and_dual_objective_optimization.ipynb")

{2187: array([[1.192912e-04, 3.261758e-04, 5.381881e-02, ..., 3.890337e-02,
         5.959047e-02, 6.730249e-02],
        [3.029030e-01, 2.900323e+00, 5.797945e+00, ..., 2.990299e+00,
         6.188119e+00, 8.188973e+00],
        [1.000074e-04, 3.520803e-03, 5.334089e-02, ..., 5.748649e-01,
         1.683214e+00, 1.864418e+00],
        ...,
        [1.192910e-04, 1.200854e-04, 1.212422e-04, ..., 1.095682e-02,
         2.717968e-02, 1.626874e-02],
        [1.192919e-04, 1.001522e-04, 2.257122e+00, ..., 7.200528e+00,
         7.815804e+00, 6.147962e+00],
        [2.131740e-01, 2.519483e+01, 1.303084e+01, ..., 1.587244e+01,
         3.931767e+01, 3.902884e+01]]),
 2411: array([[1.205633e-04, 1.306547e-04, 9.354108e-03, ..., 2.659298e-04,
         5.490244e-03, 1.090455e-02],
        [2.544912e-01, 1.737243e+00, 1.589326e+00, ..., 8.607241e-01,
         1.459218e+00, 1.456638e+00],
        [1.206061e-04, 1.199036e-04, 1.228180e-04, ..., 5.787584e-03,
         2.339844e-02, 5.644549e-02],
 

In [5]:
len(test.keys())

2921

In [157]:
file_name = "ghg.gid.site0001.dat"
file_path = os.path.join(data_folder, file_name)
with open(file_path, "r") as file:
    contents = file.read()
    lines = contents.splitlines()
    # lines = [line.strip() if line.strip() else line for line in contents]
    print(lines)

['1.203126e-04 1.211998e-04 1.185726e-04 1.007970e-04 7.808453e-04 3.268015e-03 7.286873e-03 1.881583e-02 2.978242e-02 2.613451e-02 3.249887e-02 3.550026e-02 2.365685e-02 1.799708e-02 2.658169e-02 3.487401e-02 4.158247e-02 4.180585e-02 4.085280e-02 3.272150e-02 3.811545e-02 3.945418e-02 3.766900e-02 3.487374e-02 3.448471e-02 3.676430e-02 3.533213e-02 2.990111e-02 2.477689e-02 8.167157e-03 8.402229e-03 3.227282e-04 1.276052e-04 1.823234e-04 6.030791e-04 5.022639e-04 1.878157e-03 1.512365e-03 1.170252e-03 5.602684e-04 1.190915e-04 1.196721e-04 1.202959e-04 1.193807e-03 1.352306e-04 1.184038e-04 1.207494e-04 1.222491e-04 1.203456e-04 1.209337e-04 1.210401e-04 1.197877e-04 7.375542e-04 1.138642e-02 2.919504e-02 3.394627e-03 3.677571e-03 1.015847e-03 1.257924e-03 3.786760e-04 1.109222e-04 3.003988e-04 5.183791e-04 1.173699e-04 1.215882e-04 1.215265e-04 1.211866e-04 1.208411e-04 1.212411e-04 1.211276e-04 1.212824e-04 1.213040e-04 1.208107e-04 1.208488e-04 3.475120e-04 8.665685e-03 7.535012e-

In [164]:
lines[16]

'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [149]:
s = lines[-1]
len(s)
# end_index = contents.find(b'\x00')

136239

In [153]:
len(s[-3548:].split())

273

In [143]:
# Regex pattern to match the valid float format
pattern = r"-?\d+\.\d+e[+-]0[1-9]"
# re.findall(pattern, lines[16])
" ".join(re.findall(pattern, lines[16]))

'7.502232e+01 3.752237e+01 9.353789e+01 1.594173e+02 2.069809e+02 2.359257e+02 2.632321e+02 1.888104e+02 1.189022e+02 1.115092e+02 1.101793e+02 4.380155e+01 7.134636e+01 8.637047e+01 7.569157e+01 1.402671e+02 1.812657e+02 2.478627e+02 2.870925e+02 2.106075e+02 1.202581e+02 9.558777e+01 3.884001e+01 4.842164e+01 8.103088e+01 2.077257e+01 4.497546e+01 2.496165e+01 2.541582e+01 5.733804e+01 3.237651e+01 4.035296e+01 3.576495e+01 3.009548e+01 4.781968e+01 1.821032e+01 3.096029e+01 1.542398e+01 4.203878e+01 7.866360e+01 1.007084e+02 2.556526e+02 1.971285e+02 7.286738e+01 4.458549e+01 4.074787e+01 4.092766e+01 6.351124e+01 1.566925e+02 1.584581e+02 1.403163e+02 9.393476e+01 5.548891e+01 7.638416e+01 1.423346e+02 1.136766e+02 7.207382e+01 2.742435e+01 3.802101e+01 4.089501e+01 7.469401e+01 6.000627e+01 2.685225e+01 1.145761e+01 8.201596e+01 5.561760e+01 3.073555e+01 2.841969e+01 5.547961e+01 3.165528e+01 2.300488e+01 2.365670e+01 4.072784e+01 2.189107e+01 4.397553e+01 2.830400e+01 1.516379e+0

In [87]:
print(s)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [187]:
# Regex pattern to match the valid float format
pattern = r"-?\d+\.\d+e[+-]0[1-9]"
corrupted_row_retrieved_data = " ".join(re.findall(pattern, lines[16]))

In [188]:
corrupted_row_retrieved_data

'7.502232e+01 3.752237e+01 9.353789e+01 1.594173e+02 2.069809e+02 2.359257e+02 2.632321e+02 1.888104e+02 1.189022e+02 1.115092e+02 1.101793e+02 4.380155e+01 7.134636e+01 8.637047e+01 7.569157e+01 1.402671e+02 1.812657e+02 2.478627e+02 2.870925e+02 2.106075e+02 1.202581e+02 9.558777e+01 3.884001e+01 4.842164e+01 8.103088e+01 2.077257e+01 4.497546e+01 2.496165e+01 2.541582e+01 5.733804e+01 3.237651e+01 4.035296e+01 3.576495e+01 3.009548e+01 4.781968e+01 1.821032e+01 3.096029e+01 1.542398e+01 4.203878e+01 7.866360e+01 1.007084e+02 2.556526e+02 1.971285e+02 7.286738e+01 4.458549e+01 4.074787e+01 4.092766e+01 6.351124e+01 1.566925e+02 1.584581e+02 1.403163e+02 9.393476e+01 5.548891e+01 7.638416e+01 1.423346e+02 1.136766e+02 7.207382e+01 2.742435e+01 3.802101e+01 4.089501e+01 7.469401e+01 6.000627e+01 2.685225e+01 1.145761e+01 8.201596e+01 5.561760e+01 3.073555e+01 2.841969e+01 5.547961e+01 3.165528e+01 2.300488e+01 2.365670e+01 4.072784e+01 2.189107e+01 4.397553e+01 2.830400e+01 1.516379e+0

In [186]:
np.loadtxt(corrupted_row_retrieved_data, dtype=float)

FileNotFoundError: 7.502232e+01 3.752237e+01 9.353789e+01 1.594173e+02 2.069809e+02 2.359257e+02 2.632321e+02 1.888104e+02 1.189022e+02 1.115092e+02 1.101793e+02 4.380155e+01 7.134636e+01 8.637047e+01 7.569157e+01 1.402671e+02 1.812657e+02 2.478627e+02 2.870925e+02 2.106075e+02 1.202581e+02 9.558777e+01 3.884001e+01 4.842164e+01 8.103088e+01 2.077257e+01 4.497546e+01 2.496165e+01 2.541582e+01 5.733804e+01 3.237651e+01 4.035296e+01 3.576495e+01 3.009548e+01 4.781968e+01 1.821032e+01 3.096029e+01 1.542398e+01 4.203878e+01 7.866360e+01 1.007084e+02 2.556526e+02 1.971285e+02 7.286738e+01 4.458549e+01 4.074787e+01 4.092766e+01 6.351124e+01 1.566925e+02 1.584581e+02 1.403163e+02 9.393476e+01 5.548891e+01 7.638416e+01 1.423346e+02 1.136766e+02 7.207382e+01 2.742435e+01 3.802101e+01 4.089501e+01 7.469401e+01 6.000627e+01 2.685225e+01 1.145761e+01 8.201596e+01 5.561760e+01 3.073555e+01 2.841969e+01 5.547961e+01 3.165528e+01 2.300488e+01 2.365670e+01 4.072784e+01 2.189107e+01 4.397553e+01 2.8304

In [180]:
intact_rows = np.loadtxt(lines[:-1], dtype=float)
intact_rows.shape

(16, 327)

In [184]:
corrupted_row = np.zeros(327)
avilable_data_idx = intact_rows.shape[0] - len(corrupted_row_retrieved_data)

In [185]:
avilable_data_idx

-3246

In [4]:
file_name = file_list[0]
file_path = os.path.join(data_folder, file_name)
with open(file_path, "r") as file:
    contents = file.read()
    print(contents)

1.192912e-04 3.261758e-04 5.381881e-02 2.373142e-02 9.126911e-03 1.294428e-01 1.213113e-01 6.780240e-02 2.793845e-02 1.201406e-01 2.506185e-01 1.229505e-01 3.675925e-02 1.199445e-01 9.131640e-02 5.284581e-02 4.977333e-02 9.457681e-02 7.496780e-02 5.121770e-02 5.349364e-02 4.693146e-02 3.102312e-02 2.654609e-02 1.028136e-02 4.458832e-03 1.248813e-03 1.222667e-04 1.246267e-04 1.228906e-04 1.285038e-04 1.217826e-04 1.213549e-04 1.211631e-04 1.214697e-04 1.212868e-04 1.182240e-04 1.180627e-04 1.181909e-04 1.127342e-04 1.202370e-04 2.384746e-04 1.254976e-04 1.198929e-04 1.201258e-04 1.205423e-04 1.208196e-04 1.210867e-04 1.164884e-04 8.030463e-02 1.116835e-01 2.043814e-03 1.230527e-04 1.331102e-04 1.170500e-04 1.205505e-04 1.207150e-04 1.205833e-04 1.205014e-04 1.208918e-04 1.208738e-04 1.201757e-04 1.199548e-04 1.191631e-04 1.209034e-04 1.211511e-04 1.187513e-04 1.162617e-04 1.188505e-04 1.277639e-04 2.590616e-04 3.279935e-03 8.275494e-03 2.601326e-02 1.441748e-01 9.754823e-02 9.239233e-02

In [13]:
# pd.read_csv(file_path, names=timestamps)

In [49]:
contents
np.loadtxt(contents.splitlines(), dtype=float)

ValueError: the number of columns changed from 327 to 274 at row 17; use `usecols` to select a subset and avoid this error

In [18]:
pd.DataFrame(np.loadtxt(contents.splitlines(), dtype=float))

,0,1,2,3,4,5,6,7,8,9,...,317,318,319,320,321,322,323,324,325,326
0,0.000119,0.000326,0.053819,0.023731,0.009127,0.129443,0.121311,0.067802,0.027938,0.120141,...,0.011850,0.008230,0.011758,0.016027,0.016875,0.020802,0.044683,0.038903,0.059590,0.067302
1,0.302903,2.900323,5.797945,2.373056,2.763315,4.395503,15.125510,3.139457,2.991631,5.681137,...,10.330260,16.286550,6.407364,3.684711,8.281940,7.530284,5.193272,2.990299,6.188119,8.188973
2,0.000100,0.003521,0.053341,0.000454,0.010635,0.979829,0.633246,0.005838,0.000100,0.391851,...,1.608027,1.689328,0.909500,0.860568,1.791835,2.158558,1.707953,0.574865,1.683214,1.864418
3,0.000119,0.000119,0.000167,0.000112,0.000575,0.001693,0.000878,0.000129,0.000121,0.000182,...,0.132185,0.099885,0.076240,0.062653,0.117395,0.112509,0.086171,0.050053,0.112303,0.098606
4,0.000100,0.000100,0.003904,0.000100,0.000338,0.202054,0.255714,0.000100,0.000100,0.005997,...,0.593305,1.807263,0.202013,0.090686,0.223497,0.207715,0.082819,0.012771,0.089157,0.111505
5,0.000119,0.000120,0.000121,0.000122,0.000122,0.000121,0.000121,0.000122,0.000122,0.000122,...,0.000466,0.000344,0.000317,0.000282,0.000394,0.000366,0.000325,0.000344,0.000512,0.000397
6,0.000119,0.000155,0.000174,0.000150,0.005215,0.017301,0.007372,0.000135,0.000122,0.000132,...,9.806330,7.143551,4.854272,3.218907,6.895373,6.309807,3.403048,0.939852,3.457914,2.670069
7,0.000119,0.000121,0.000120,0.000114,0.000464,0.001277,0.000619,0.000122,0.000122,0.000122,...,2.036406,1.376209,1.038793,0.659500,1.212258,0.991146,0.558086,0.203738,0.678581,0.397235
8,0.000119,0.000120,0.000120,0.000120,0.000121,0.000120,0.000119,0.000122,0.000122,0.000122,...,1.272406,0.911740,0.660369,0.379099,0.722015,0.618024,0.334093,0.103674,0.381023,0.232137
9,0.000119,0.000120,0.000121,0.000122,0.000122,0.000122,0.000121,0.000122,0.000122,0.000122,...,0.001404,0.001007,0.000910,0.000701,0.001084,0.000960,0.000795,0.000926,0.001543,0.001111


In [7]:
def get_files(path: str) -> str:
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file

In [20]:
pd.read_table(
    "/home/mghanava/DataScience/projects/optimal_gas_observing_networks/data/ghg.gid.site0001.dat"
)

1.203126e-04 1.211998e-04 1.185726e-04 1.007970e-04 7.808453e-04 3.268015e-03 7.286873e-03 1.881583e-02 2.978242e-02 2.613451e-02 3.249887e-02 3.550026e-02 2.365685e-02 1.799708e-02 2.658169e-02 3.487401e-02 4.158247e-02 4.180585e-02 4.085280e-02 3.272150e-02 3.811545e-02 3.945418e-02 3.766900e-02 3.487374e-02 3.448471e-02 3.676430e-02 3.533213e-02 2.990111e-02 2.477689e-02 8.167157e-03 8.402229e-03 3.227282e-04 1.276052e-04 1.823234e-04 6.030791e-04 5.022639e-04 1.878157e-03 1.512365e-03 1.170252e-03 5.602684e-04 1.190915e-04 1.196721e-04 1.202959e-04 1.193807e-03 1.352306e-04 1.184038e-04 1.207494e-04 1.222491e-04 1.203456e-04 1.209337e-04 1.210401e-04 1.197877e-04 7.375542e-04 1.138642e-02 2.919504e-02 3.394627e-03 3.677571e-03 1.015847e-03 1.257924e-03 3.786760e-04 1.109222e-04 3.003988e-04 5.183791e-04 1.173699e-04 1.215882e-04 1.215265e-04 1.211866e-04 1.208411e-04 1.212411e-04 1.211276e-04 1.212824e-04 1.213040e-04 1.208107e-04 1.208488e-04 3.475120e-04 8.665685e-03 7.535012e-03 1.779663e-02 9.806178e-03 1.658397e-02 2.360511e-02 1.309094e-02 1.515201e-02 9.694790e-03 1.099980e-02 2.438278e-02 1.531150e-02 1.436246e-02 2.759920e-02 2.990931e-02 2.886357e-02 2.378380e-02 2.204072e-02 1.508766e-02 6.048019e-03 9.429220e-03 2.189632e-03 2.100577e-03 2.625715e-03 3.305885e-03 1.285039e-03 1.186657e-03 1.132667e-03 1.755603e-03 1.820842e-03 1.425502e-03 1.432477e-03 1.900549e-03 8.899939e-04 6.952566e-04 6.413477e-04 3.073443e-04 3.514219e-04 4.354908e-04 4.546014e-04 6.298118e-04 5.452496e-03 6.699682e-03 1.000003e-04 1.000122e-04 1.031320e-04 1.144440e-04 1.194611e-04 1.185351e-04 1.190917e-04 1.191674e-04 1.214663e-04 1.219342e-03 2.804016e-02 5.844307e-02 6.247029e-02 4.847275e-02 3.724229e-02 3.618167e-02 3.587751e-02 3.527120e-02 3.405875e-02 3.098921e-02 3.068760e-02 4.500770e-02 2.617079e-02 2.635155e-02 2.558327e-02 1.976736e-02 8.162502e-03 5.332826e-03 1.076794e-02 1.953674e-02 1.116611e-02 1.078132e-02 1.050511e-02 1.164551e-02 2.245865e-02 1.616556e-02 1.598640e-02 2.498590e-02 1.242738e-02 1.255994e-02 1.229189e-02 3.984907e-03 2.936275e-03 7.271928e-04 1.758482e-03 2.719123e-04 3.287794e-04 5.826224e-04 2.867437e-03 1.222941e-02 1.304620e-02 9.583720e-03 8.627821e-03 9.140680e-03 9.403354e-03 2.232614e-02 2.218642e-02 2.299228e-02 4.502853e-02 3.554804e-02 3.445316e-02 4.919681e-02 4.401325e-02 3.623728e-02 3.726416e-02 1.845219e-03 2.710505e-03 1.437500e-02 1.702778e-02 2.322367e-02 1.774198e-02 1.981768e-02 1.873222e-02 1.167516e-02 1.002133e-02 1.335523e-02 1.583366e-02 1.712435e-02 1.760116e-02 1.402279e-02 1.477895e-02 1.416257e-02 1.162255e-02 1.225039e-02 1.179976e-02 1.008377e-04 1.026938e-04 4.175809e-04 3.313052e-03 8.661417e-03 8.913438e-03 3.675337e-03 1.181524e-02 1.000725e-02 5.741711e-03 6.288174e-03 4.573489e-03 1.249075e-03 6.242666e-04 6.648296e-04 8.014905e-04 6.059224e-04 5.154290e-04 1.436867e-03 1.658801e-03 3.082274e-03 1.597729e-02 2.134112e-02 1.000001e-04 1.000002e-04 1.000002e-04 1.000001e-04 1.000126e-04 1.000291e-04 1.000432e-04 1.000258e-04 1.003822e-04 1.012579e-04 1.028690e-04 1.267313e-04 1.633850e-04 4.051317e-03 4.913027e-03 6.696963e-03 8.020733e-03 6.659708e-03 2.267386e-02 4.726004e-02 3.852146e-02 2.558125e-02 2.623597e-02 2.224455e-02 1.831670e-02 2.224141e-02 2.762551e-02 9.229169e-03 8.222333e-03 9.238010e-03 9.490920e-03 2.015319e-02 2.436620e-02 2.724098e-02 2.592034e-02 2.645283e-02 1.854956e-02 1.455810e-02 1.180788e-02 1.261994e-02 1.259809e-02 1.131484e-02 9.259907e-03 1.065324e-02 9.906376e-03 1.817243e-02 1.767320e-02 1.488472e-02 1.007600e-02 9.850790e-03 6.618313e-03 2.807990e-03 2.161225e-03 4.620184e-03 4.272041e-03 4.229001e-03 4.287593e-03 3.966596e-03 4.558913e-03 6.087863e-03 1.009272e-04 1.022699e-04 1.013376e-04 1.014958e-04 1.055581e-04 6.142929e-04 5.772735e-03 1.280488e-02 1.263867e-02 1.575149e-02 1.747722e-02 1.990849e-02 8.295249e-03 3.299997e-03 2.762398e-03 9.352110e-03 1.056777e-02 1.941146e-02 1.329705e-02 1.479319e-02 1.504330e-02 1.235124e